In [ ]:
import warnings
warnings.filterwarnings('ignore')

import glob
import pandas as pd
import numpy as np
import catboost as cb

from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
import random

import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X_train_sample= pd.read_csv('/content/drive/MyDrive/ITStudy/딥러닝 미니프로젝트/X_train_sample.csv', encoding = 'euc-kr', index_col = 0)
y_train_sample= pd.read_csv('/content/drive/MyDrive/ITStudy/딥러닝 미니프로젝트/y_train_sample.csv', encoding = 'euc-kr', index_col = 0)
X_test_sample= pd.read_csv('/content/drive/MyDrive/ITStudy/딥러닝 미니프로젝트/X_test_sample.csv', encoding = 'euc-kr', index_col = 0)
y_test_sample= pd.read_csv('/content/drive/MyDrive/ITStudy/딥러닝 미니프로젝트/y_test_sample.csv', encoding = 'euc-kr', index_col = 0)

##index 밀어냄

In [ ]:
X_train_sample3 = X_train_sample.loc[:247036,:].reset_index(drop = True)
y_train_sample3 = y_train_sample.loc[1:,:].reset_index(drop = True)

X_test_sample3 = X_test_sample.loc[:253009, :].reset_index(drop = True)
y_test_sample3 = y_test_sample.loc[247039:, ].reset_index(drop = True)

In [ ]:
X_train1 = X_train_sample3.copy()
y_train1 = y_train_sample3.copy()

In [ ]:
y2 = y_train1.drop(['한강대교 수위 (단위: cm)','청담대교 수위 (단위: cm)','잠수교 수위 (단위: cm)'],axis =1)    # 행주대교    타겟값을 해당 칼럼만 지정.
##숫자들은 대교별 구별을 위해 끝에 붙여준 것 입니다. 숫자만 수정해주시고 돌리면 됩니다.

In [ ]:
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train1, y2, test_size=0.2, random_state=42)      # 20%만 X,y데이터로 나누기            
##숫자들은 대교별 구별을 위해 끝에 붙여준 것 입니다. 숫자만 수정해주시고 돌리면 됩니다.

In [ ]:
# random sampler
sampler = TPESampler(seed=10)

# define function
def objective(trial):

    cbrm_param = {
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }

    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param)
    model_cbrm = model_cbrm.fit(X_train2, y_train2, eval_set=[(X_val2, y_val2)], 
                           verbose=0, early_stopping_rounds=25)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    MSE = mean_squared_error(y_val2, model_cbrm.predict(X_val2))
    return MSE

optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=45)